In [20]:
#Import all the required packages
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D,Dense,BatchNormalization,Dropout
from tensorflow.keras.layers import Flatten
from keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from jeanCV import skinDetector
import cv2
import os
import numpy as np

In [10]:
#Model definition
model = Sequential()
model.add(Conv2D(32, kernel_size = (5, 5), activation='relu',kernel_regularizer=l2(0.001), input_shape=(670, 600, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(5,5), kernel_regularizer=l2(0.001),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(5,5), kernel_regularizer=l2(0.001),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(5,5), kernel_regularizer=l2(0.001),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(5,5), kernel_regularizer=l2(0.001),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(36, activation = 'softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

In [11]:
#load the weights that were saved after training the model
model.load_weights("finalweights.h5")

In [13]:
# Load the testing image here
# Replace filename.xyz with your file name and extention
# Put the associated file in the present working directory
img=cv2.imread(os.path.join('./',"hand3.png"))
detector = skinDetector(img)
im = detector.find_skin()  #Skin segmentation is being done here which uses functions from jeanCV.ipynb file

In [14]:
img=np.array(im)
img=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

In [15]:
#resizing the image to nearest multiples of height 670 and width 600
#so as to maintain the aspect ratio and the resize to 670*600
for i in range(670):
    if((i+img.shape[0])%670==0):
        break
for j in range(600):
    if((j+img.shape[1])%600==0):
        break
img=np.pad(img,((int(i/2),int(i/2)),(int(j/2),int(j/2))),'constant',constant_values=0)
if(i%2!=0):
    img=np.pad(img,((1,0),(0,0)),'constant',constant_values=0)
if(j%2!=0):
    img=np.pad(img,((0,0),(1,0)),'constant',constant_values=0)

In [16]:
img1=cv2.resize(img,dsize=(600,670))

In [17]:
#Predicting the output for the required input image
pred_y=model.predict_classes(img1.reshape(1,670,600,1))

In [18]:
#Display the result
if(pred_y[0]<=9):
    print(pred_y[0])
else:
    print(chr(pred_y[0]+55))

F
